In [1]:
import numpy as np　#数値計算統計解析やデータ分析などで、 データ加工や演算をスムーズに行うためのライブラリ
import matplotlib.pyplot as plt　#暗黙的かつ自動的に図形や軸を作成するインターフェース
import pandas as pd #データの読込や並べ替え、欠損値（欠けている項目値）の補完などを行うことができるライブラリ
import random #ランダムな要素や乱数を使う
%matplotlib inline
import seaborn as sns #データの可視化(グラフ作成)を行うPythonライブラリです。 Pythonで有名な可視化ライブラリであるmatplotlibのラッパー関数(内包プログラム)

import requests #HTTP通信ライブラリ
import json　#jsonファイルからの読み込み、データの文字列 ⇔ 辞書型変換、ファイルへの保存を行う
import re #正規表現を行うモジュール





a


In [ ]:
# APIの仕様は
url_path = "https://www.land.mlit.go.jp/webland/api/TradeListSearch?from=20171&to=20175&area=13"
request_result = requests.get(url_path)
data_json = request_result.json()["data"]